In [1]:
import ipywidgets
from pathlib import Path

In [2]:
# config 
bar_width = 150
title_height = 100
favorite_folders = ["/Users/niwn/ipywidget-ws/","/Users/niwn/Downloads/"]

In [3]:
ipywidgets.Select(
    options=["a","b","c"]
)

Select(options=('a', 'b', 'c'), value='a')

In [15]:
# Utils widgets
from ipywidgets import Button, Layout, jslink, IntText, IntSlider
from ipywidgets import GridspecLayout

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

top_left_text = IntText(description='Top left', layout=Layout(width='auto', height='auto'))
top_right_text = IntText(description='Top right', layout=Layout(width='auto', height='auto'))
bottom_left_slider = IntSlider(description='Bottom left', layout=Layout(width='auto', height='auto'))
bottom_right_slider = IntSlider(description='Bottom right', layout=Layout(width='auto', height='auto'))

In [16]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

In [17]:
from ipywidgets import Layout, Button, Box, HBox, VBox, Select
from typing import Union

In [18]:
from ipywidgets import Layout, Button, VBox, Label

In [24]:
from traitlets.utils.bunch import Bunch
import ipywidgets as widgets
from ipywidgets import Output
from IPython.display import display

In [155]:
def generate_selector(folder_path:Union[Path,str], callback = lambda :None,  **kwargs) -> Select:
    """"""
    if Path(folder_path).is_dir():
        options = [
            path.stem + ("/" if path.is_dir() else "")
            for path in 
            Path(folder_path).iterdir()
        ]
    else:
        options=[""]
    select = Select(
        options = options,
        **kwargs
    )
    select.observe(callback)
    return select
generate_selector(".")

Select(options=('tab_explorer', 'Untitled1', 'make-gui', 'Untitled-1', 'README', 'Pipfile', '.git/', '.vscode/…

In [156]:
from pprint import pprint

In [157]:
# ハンドラを変更することで対応するようにしたい
# on_value_changeにステータスIDかなんかを渡す

In [158]:

from functools import partial
def list_dir(path)->tuple:
    if Path(path).is_dir():
        return tuple(
            i.name+"/" if i.is_dir() else i.name
            for i 
            in Path(path).iterdir()
        )
    else:
        return [""]
         
def generate_select(path):
    return widgets.Select(
        options=list_dir(path),
        disabled=False,
        rows=10,
        layout = Layout(min_width="150px",width="150px")
    )

paths = [".", ".git", ".git/info"]
curdir_dict = {i:Path(path) for i,path in enumerate(paths)}
select_objects = list()
for i, path in enumerate(paths):
    select_objects.append(generate_select(path))

output = widgets.Output(layour={'border': '1px solid black'})

@output.capture(clear_output=True)
def update_child(changes, root_index): # changesが第一変数で入ってくることに気づかず苦戦した
    with output:
        print(root_index, changes)
    # 自分の場所に応じて
    # 階層+1を更新して+2以降を初期化する
    curdir = curdir_dict[root_index] 
    select = grid[:, root_index]
    next_path = curdir / select.get_interact_value()

    # grid[:, root_index+1].unobserve(handlers[root_index+1])
    # if next_path.is_dir():
    #     grid[:, root_index+1].options=list_dir(next_path)
    # else:
    #     grid[:, root_index+1].options=[f"{next_path} is not dir"]
    # grid[:, root_index+1:].close()
    
    grid[:, root_index+1] = generate_select(next_path)
    grid[:, root_index+1].observe(handlers[root_index+1])
    curdir_dict[root_index+1] = next_path

handlers = [partial(update_child, root_index=i) for i in range(len(paths))]

In [159]:
# gridを作成して登録する
grid = GridspecLayout(1, 3)
for i, select in enumerate(select_objects):
    grid[:, i] = select
    grid[:, i].observe(handlers[i])
display(grid, output)

GridspecLayout(children=(Select(layout=Layout(grid_area='widget001', min_width='150px', width='150px'), option…

Output()

In [162]:
class SelectHandler():
    pass

output = widgets.Output(layour={'border': '1px solid black'})

class Explorer(HBox):
    def __init__(self):
        # 左のサイドバー部分
        sidebar = VBox(
            [create_expanded_button(Path(folder_path).stem+"/","info") for folder_path in favorite_folders],
            layout=Layout(width="150px", display = 'flex', align_items = 'stretch',flex_flow = 'column'),
            height="100%")
        
        # タイトルボックス、現在のパスを描画する部分
        title_box = Output(layout=Layout(border= '1px solid black',height="100px"))
        print(".")

        @title_box.capture()
        def on_value_change(change: Bunch) -> None:
            # 値が変更されたイベントを扱う
            # label -> value -> indexの順番に変化する
            if change['name'] == 'value':
                title_box.clear_output()
                # 変更前と変更後の値を出力する
                old_value = change['old']
                new_value = change['new']
                print(f'value changed: {old_value} -> {new_value}')
                pprint(f'value changed: {change}')

        # folder explorerの部分
        self.item_layout = Layout(min_width="150px",width="150px")
        self.paths = [Path("."),Path(".git"),Path(".git/info")]
        self.handlers = [partial(self._update_child, root_index=i) for i,path in enumerate(self.paths)]
        items = [generate_selector(path, callback = self.handlers[i], rows=12, layout=self.item_layout) for i,path in enumerate(self.paths)]
        box_layout = Layout(overflow='scroll hidden',
            width='auto',
            height='100%',
            flex_flow='row',
            display='flex')
        self.carousel = Box(children=items, layout=box_layout)
        # folder_content = VBox([carousel], layout=Layout(height='100%', width='100%'))

        right_box = VBox([title_box, self.carousel],layout=Layout(width="100%", display = 'flex', align_items = 'stretch',flex_flow = 'column',height="100%"))

        super(HBox, self).__init__([sidebar,right_box],layout=Layout(width="100%", display = 'flex', align_items = 'stretch',height="300px"))

    @output.capture()
    def _update_child(self, changes, root_index): # changesが第一変数で入ってくることに気づかず苦戦した

        # 自分の階層より下がある場合まず削除する
        if len(self.carousel.children) > root_index + 1:
            for i in reversed(range(root_index+1, len(self.carousel.children))):
                with output:
                    # print(root_index, changes)
                    print(i)
                child = self.carousel.children[i]
                # child.unobserve(self.handlers[i])
                child.close()
            # del self.handlers[root_index+1:]
            del self.paths[root_index+1:]

        # 自分の場所に応じて
        # 階層+1を更新して+2以降を初期化する
        curdir = self.paths[root_index]
        select = self.carousel.children[root_index]
        next_path = curdir / select.get_interact_value()

        self.carousel.children += (generate_selector(next_path, callback = self.handlers[root_index+1], rows=12, layout=self.item_layout),)
        self.carousel.children[-1].observe(partial(self._update_child, root_index=root_index+1))
        self.paths.append(next_path)

explorer = Explorer()
display(explorer, output)

.


Explorer(children=(VBox(children=(Button(button_style='info', description='ipywidget-ws/', layout=Layout(heigh…

Output()

In [146]:
explorer.handlers

[functools.partial(<bound method Explorer._update_child of Explorer(children=(VBox(children=(Button(button_style='info', description='ipywidget-ws/', layout=Layout(height='auto', width='auto'), style=ButtonStyle()), Button(button_style='info', description='Downloads/', layout=Layout(height='auto', width='auto'), style=ButtonStyle())), layout=Layout(align_items='stretch', display='flex', flex_flow='column', width='150px')), VBox(children=(Output(layout=Layout(border='1px solid black', height='100px')), Box(children=(Select(layout=Layout(min_width='150px', width='150px'), options=('tab_explorer', 'Untitled1', 'make-gui', 'Untitled-1', 'README', 'Pipfile', '.git/', '.vscode/', 'Pipfile', 'x'), rows=12, value='tab_explorer'), Select(layout=Layout(min_width='150px', width='150px'), options=('ORIG_HEAD', 'config', 'objects/', 'HEAD', 'info/', 'logs/', 'description', 'hooks/', 'refs/', 'index', 'packed-refs', 'COMMIT_EDITMSG', 'FETCH_HEAD'), rows=12, value='ORIG_HEAD'), Select(layout=Layout(m

In [131]:
explorer.carousel.children

(Select(layout=Layout(min_width='150px', width='150px'), options=('tab_explorer', 'Untitled1', 'make-gui', 'Untitled-1', 'README', 'Pipfile', '.git/', '.vscode/', 'Pipfile', 'x'), rows=12, value='tab_explorer'),
 Select(index=5, layout=Layout(min_width='150px', width='150px'), options=('ORIG_HEAD', 'config', 'objects/', 'HEAD', 'info/', 'logs/', 'description', 'hooks/', 'refs/', 'index', 'packed-refs', 'COMMIT_EDITMSG', 'FETCH_HEAD'), rows=12, value='logs/'),
 Select(layout=Layout(min_width='150px', width='150px'), options=('exclude',), rows=12, value='exclude'),
 Select(layout=Layout(min_width='150px', width='150px'), options=('',), rows=10, value=''),
 Select(layout=Layout(min_width='150px', width='150px'), options=('',), rows=10, value=''),
 Select(layout=Layout(min_width='150px', width='150px'), options=('',), rows=10, value=''),
 Select(layout=Layout(min_width='150px', width='150px'), options=('68/', '04/', 'bc/', 'f5/', 'pack/', '45/', '21/', '38/', '00/', 'info/', '62/', 'b0/', 